In [1]:
%pip install pyspark


  Using cached pyspark-3.5.5-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)

   ---------------------------------------- 0/2 [py4j]
   ---------------------------------------- 0/2 [py4j]
   ---------------------------------------- 0/2 [py4j]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   ------------------

In [2]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkContext

In [ ]:
c = SparkContext.getOrCreate()
spark = SparkSession.builder /
    .appName("Finding strategies for dataframe transformation") /
    .master("local[*]") /
    .getOrCreate()
    


In [6]:
airpods = spark.read.parquet("C:/Users/macmi/Desktop/Prj/ELT-prj/data/airpods.parquet")
airpods.printSchema()

root
 |-- itemId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- leafCategoryIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- categoryId: string (nullable = true)
 |    |    |-- categoryName: string (nullable = true)
 |-- image: struct (nullable = true)
 |    |-- imageUrl: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- value: string (nullable = true)
 |-- itemHref: string (nullable = true)
 |-- seller: struct (nullable = true)
 |    |-- feedbackScore: long (nullable = true)
 |-- condition: string (nullable = true)
 |-- conditionId: string (nullable = true)
 |-- thumbnailImages: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- imageUrl: string (nullable = true)
 |-- shippingOptions: array (nullable = true)
 |    |-- element: struct

In [7]:
airpods.show(1)

+-----------------+--------------------+---------------+--------------------+--------------------+------------+--------------------+------+---------+-----------+--------------------+--------------------+-------------+--------------------+------------+---------+------------+----------------+--------------------+--------------------+------------------------+---------------+--------------------+
|           itemId|               title|leafCategoryIds|          categories|               image|       price|            itemHref|seller|condition|conditionId|     thumbnailImages|     shippingOptions|buyingOptions|          itemWebUrl|itemLocation|adultOnly|legacyItemId|availableCoupons|      itemOriginDate|    itemCreationDate|topRatedBuyingExperience|priorityListing|listingMarketplaceId|
+-----------------+--------------------+---------------+--------------------+--------------------+------------+--------------------+------+---------+-----------+--------------------+--------------------+-----

In [8]:
new_airpod_df = airpods.select('itemId', 'title', 'price')

In [9]:
new_airpod_df.printSchema()

root
 |-- itemId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- value: string (nullable = true)



In [10]:
from pyspark.sql.functions import col

# Ép price.value -> double và tạo cột mới "price"
df2 = new_airpod_df.withColumn("price", col("price.value").cast("double"))

# Nếu bạn muốn bỏ luôn struct cũ "price", thì chọn lại cột
final_df = df2.select("itemId", "title", "price")

In [11]:
from pyspark.sql.functions import lit

final_df = final_df.withColumn("brand_name", lit("ebay"))

In [12]:
from pyspark.sql.functions import current_timestamp
final_df = final_df.withColumn("first_issue_date", current_timestamp())

In [13]:
final_df.show(1)

+-----------------+--------------------+-----+----------+--------------------+
|           itemId|               title|price|brand_name|    first_issue_date|
+-----------------+--------------------+-----+----------+--------------------+
|v1|110582305802|0|Apple Airpods 2nd...|59.99|      ebay|2025-04-26 23:10:...|
+-----------------+--------------------+-----+----------+--------------------+
only showing top 1 row



In [19]:
final_df = final_df.withColumnRenamed('itemId', 'product_id').withColumnRenamed('title', 'product_name')


In [20]:
final_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_name: string (nullable = false)
 |-- first_issue_date: timestamp (nullable = false)

